<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/Econophysics/NR7%3FStocksIB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pandas_ta
!pip install ib_insync
# Import required libraries
import pandas as pd
import numpy as np
import pandas_ta as ta
from ib_insync import IB, Stock, util, BarDataList

# Define the stocks to be analyzed
stocks = ['AAPL', 'GOOG', 'AMZN', 'MSFT']

# Define the start and end date of the historical data
start_date = '2010-01-01'
end_date = '2022-04-16'

# Connect to the Interactive Brokers API
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)

# Retrieve historical price data for the stocks from Interactive Brokers
prices = pd.DataFrame()
for stock_symbol in stocks:
    try:
        contract = Stock(stock_symbol, 'SMART', 'USD')
        ib.qualifyContracts(contract)
        bars = ib.reqHistoricalData(contract, endDateTime=end_date, durationStr='12 Y', barSizeSetting='1 day', whatToShow='TRADES', useRTH=True)
        data = util.df(bars)
        if 'close' in data.columns:
            prices[stock_symbol] = data.set_index('date')['close']
        else:
            print(f"Ooops!!! It's True, the series 'close' was not found in {stock_symbol}")
    except Exception as e:
        print(f"Error while fetching data for {stock_symbol}: {e}")

ib.disconnect()

# ... (The rest of the code remains the same)


# Compute the high-low range for each day over the last seven days
highs = prices.rolling(7).max()
lows = prices.rolling(7).min()
ranges = highs - lows
nr7 = ranges.iloc[-1].sort_values()

# Calculate additional technical indicators using pandas_ta
sma50 = prices.ta.sma(length=50)
sma200 = prices.ta.sma(length=200)
rsi = prices.ta.rsi(length=14)
try:
    macd = prices.ta.macd(fast=12, slow=26, signal=9)
    macdh = macd['MACDh']
except KeyError:
    macdh = pd.Series(np.nan, index=prices.index)
    print("Unable to calculate MACD")

# Perform fundamental analysis using yfinance
fundamentals = pd.DataFrame()
for stock in stocks:
    info = yf.Ticker(stock).info
    eps_growth = info.get('forwardEpsGrowth')
    pe_ratio = info.get('trailingPE')
    roe = info.get('returnOnEquity')
    fundamentals.loc['EPS Growth (next year)', stock] = eps_growth
    fundamentals.loc['PE Ratio', stock] = pe_ratio
    fundamentals.loc['Return on Equity (TTM)', stock] = roe

# Combine technical and fundamental analyses to generate a final list of potential trades
trades = pd.DataFrame(index=nr7.index)
trades['Close'] = prices.iloc[-1]
trades['50-day SMA'] = sma50.iloc[-1]
trades['200-day SMA'] = sma200.iloc[-1]
trades['RSI'] = rsi.iloc[-1]
trades['MACD'] = macdh.iloc[-1]
trades = trades.join(fundamentals)
trades = trades.dropna()

# Display the potential trades
print(trades)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


RuntimeError: ignored